In [1]:
import sys
from pyspark.sql import functions as F
from pyspark.sql.functions import * 
from awsglue.job import Job
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from functools import reduce
from pyspark.sql.types import StringType
import numpy as np
import pandas as pd
#from openpyxl.workbook import Workbook
sys.argv += ['--JOB_NAME','Custom-Translation-Trucape-Invoices-2']
sys.argv += ['--source_csv','s3://sol-dev-source/TruCape-Invoices']
args = getResolvedOptions(sys.argv, ['JOB_NAME', 'source_csv'])
sc = SparkContext.getOrCreate()
gc = GlueContext(sc)
spark = gc.spark_session
job = Job(gc)
job.init(args["JOB_NAME"])

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6,application_1644325409033_0007,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# source_dyf=gc.create_dynamic_frame.from_catalog(
#     database="s3",
#     table_name=args["source_csv"],
#     transformations_ctx="source_dyf"
# )

source_dyf = gc.create_dynamic_frame.from_options(
    's3',
    {
        "paths": [
            args["source_csv"]
        ],
    },
    "csv",
    transformation_ctx = "source_dyf",
    headerText = True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
source_df = source_dyf.toDF()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
source_df.show()


In [4]:
# (b) Create array with desired columns
old_columns = source_df.schema.names
new_columns = [
    field.lower().replace("col0", "External Reference")
    .replace("col1", "Number")
    .replace("col2", "Counterparty")
    .replace("col3", "Amount Due")
    .replace("col4", "Currency")
    .replace("col5", "Cost Currency")
    .replace("col6", "Costing Rate")
    .replace("col7", "Due Date")
    .replace("col8", "Issue Date")
    for field in old_columns
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# (c) Overwrite and persist `new_columns`
source_df = reduce(
    lambda source_df, idx: source_df.withColumnRenamed(old_columns[idx], new_columns[idx]),
    range(len(old_columns)),
    source_df,
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
source_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------+-------------------+------------+--------+-------------+------------+--------+----------+
|External Reference|Number|       Counterparty|  Amount Due|Currency|Cost Currency|Costing Rate|Due Date|Issue Date|
+------------------+------+-------------------+------------+--------+-------------+------------+--------+----------+
|External Reference|Number|       Counterparty| Amount Due |Currency|Cost Currency|Costing Rate|Due Date|Issue Date|
|           SC25648|138099|         JSC TANDER|  21,124.80 |     USD|          ZAR|     15.5654|   7-Apr|    21-Jan|
|           SC25646|138127|HORIZON FRESH FRUIT|  17,100.00 |     USD|          ZAR|     15.5142|  17-Mar|    21-Jan|
|           SC25645|138129|HORIZON FRESH FRUIT|  17,100.00 |     USD|          ZAR|     15.5142|  17-Mar|    21-Jan|
+------------------+------+-------------------+------------+--------+-------------+------------+--------+----------+

In [7]:
pandas_source_df = source_df.toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
source_df = pandas_source_df.iloc[1: , :]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
source_df=spark.createDataFrame(source_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
source_df.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------+-------------------+-----------+--------+-------------+------------+--------+----------+
|External Reference|Number|       Counterparty| Amount Due|Currency|Cost Currency|Costing Rate|Due Date|Issue Date|
+------------------+------+-------------------+-----------+--------+-------------+------------+--------+----------+
|           SC25648|138099|         JSC TANDER| 21,124.80 |     USD|          ZAR|     15.5654|   7-Apr|    21-Jan|
|           SC25646|138127|HORIZON FRESH FRUIT| 17,100.00 |     USD|          ZAR|     15.5142|  17-Mar|    21-Jan|
|           SC25645|138129|HORIZON FRESH FRUIT| 17,100.00 |     USD|          ZAR|     15.5142|  17-Mar|    21-Jan|
+------------------+------+-------------------+-----------+--------+-------------+------------+--------+----------+

In [11]:
# Add notes column to dataset
source_df = source_df.withColumn('Notes', lit(None).cast(StringType()))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
source_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------+-------------------+-----------+--------+-------------+------------+--------+----------+-----+
|External Reference|Number|       Counterparty| Amount Due|Currency|Cost Currency|Costing Rate|Due Date|Issue Date|Notes|
+------------------+------+-------------------+-----------+--------+-------------+------------+--------+----------+-----+
|           SC25648|138099|         JSC TANDER| 21,124.80 |     USD|          ZAR|     15.5654|   7-Apr|    21-Jan| null|
|           SC25646|138127|HORIZON FRESH FRUIT| 17,100.00 |     USD|          ZAR|     15.5142|  17-Mar|    21-Jan| null|
|           SC25645|138129|HORIZON FRESH FRUIT| 17,100.00 |     USD|          ZAR|     15.5142|  17-Mar|    21-Jan| null|
+------------------+------+-------------------+-----------+--------+-------------+------------+--------+----------+-----+

In [13]:
# ADD YEAR TO DATE COLUMNS
due_date="due date"
issue_date="issue date"
year = F.lit("-2022")
due_date_add_year = F.concat(col(due_date),year)
issue_date_add_year = F.concat(col(issue_date),year)
                               
source_df = source_df.withColumn(due_date, due_date_add_year) \
    .withColumn(issue_date, issue_date_add_year)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# SET DECIMAL PLACES
costing_rate="costing rate"
format_costing_rate = format_number(round(costing_rate ,4), 4)
amount_due="amount due"
format_amount_due=format_number(round(amount_due ,2), 2)

source_df = source_df.withColumn(costing_rate, format_costing_rate) \
# amount due gives null value - not sure why


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# FORMAT DATES
source_df = source_df.select(
    col("External Reference"), \
    col("Number"), \
    col("Counterparty"), \
    col("Amount due"), \
    col("Currency"), \
    col("Cost Currency"), \
    col("Costing Rate"), \
    to_date(F.col("due date"), "d-MMM-yyy").alias("Due Date"), \
    to_date(F.col("issue date"), "d-MMM-yyy").alias("Issue Date"), \
     col("Notes")
) \

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
source_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------+-------------------+-----------+--------+-------------+------------+----------+----------+-----+
|External Reference|Number|       Counterparty| Amount due|Currency|Cost Currency|Costing Rate|  Due Date|Issue Date|Notes|
+------------------+------+-------------------+-----------+--------+-------------+------------+----------+----------+-----+
|           SC25648|138099|         JSC TANDER| 21,124.80 |     USD|          ZAR|     15.5654|2022-04-07|2022-01-21| null|
|           SC25646|138127|HORIZON FRESH FRUIT| 17,100.00 |     USD|          ZAR|     15.5142|2022-03-17|2022-01-21| null|
|           SC25645|138129|HORIZON FRESH FRUIT| 17,100.00 |     USD|          ZAR|     15.5142|2022-03-17|2022-01-21| null|
+------------------+------+-------------------+-----------+--------+-------------+------------+----------+----------+-----+

In [ ]:
pandas_source_df = source_df.toPandas()

In [ ]:
GFG = pd.ExcelWriter(source_df)
pandas_source_df.to_excel(GFG, index = False)
GFG.save()

In [ ]:
#  pandas_source_df= pandas_source_df.to_excel (r's3://sol-dev-output/TruCape-Invoices/TRU_Invoices.xlsx',
# index = None, header=True)

In [ ]:
# source_df.write.mode(SaveMode.Overwrite).json('s3://sol-dev-output/TruCape-Invoices')

In [17]:
source_df.coalesce(1).write.csv(path='s3://sol-dev-output/TruCape-Invoices', mode='overwrite', header='true')

myPath = 's3://sol-dev-output/TruCape-Invoices/*'

hadoopPath = SparkContext._jvm.org.apache.hadoop.fs.Path(myPath)

hadoopFs = hadoopPath.getFileSystem(SparkContext._jvm.org.apache.hadoop.conf.Configuration())

statuses = hadoopFs.globStatus(hadoopPath)

file_name = [file.getPath().getName() for file in statuses if file.getPath().getName().startswith('part-')][0]

hadoopFs.rename(SparkContext._jvm.org.apache.hadoop.fs.Path(f"s3://sol-dev-output/TruCape-Invoices/{file_name}"), SparkContext._jvm.org.apache.hadoop.fs.Path("s3://sol-dev-output/TruCape-Invoices/TRU_Invoices.csv"))



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True

In [ ]:
# source_df.coalesce(1).write.format("json").mode("overwrite").options(header="true",sep=",").save(path='s3://sol-dev-output/TruCape-Invoices')

In [ ]:
# source_df.repartition(1).write.mode('append').json('s3://sol-dev-output/TruCape-Invoices')

# myPath = 's3://sol-dev-output/TruCape-Invoices/*'

# timeStamp = current_timestamp()

# hadoopPath = SparkContext._jvm.org.apache.hadoop.fs.Path(myPath)

# hadoopFs = hadoopPath.getFileSystem(SparkContext._jvm.org.apache.hadoop.conf.Configuration())

# statuses = hadoopFs.globStatus(hadoopPath)

# file_name = [file.getPath().getName() for file in statuses if file.getPath().getName().startswith('part-')][0]

# hadoopFs.rename(SparkContext._jvm.org.apache.hadoop.fs.Path(f"s3://sol-dev-output/TruCape-Invoices/{file_name}"), SparkContext._jvm.org.apache.hadoop.fs.Path("s3://sol-dev-output/TruCape-Invoices/TRU_Invoices.json"))


